In [7]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np

In [8]:
Tongji_375_data = pd.read_csv("data/Tongji_375_CN.csv")

In [9]:
Tongji_375_data = Tongji_375_data.dropna()

In [11]:
y = np.array(Tongji_375_data['outcome'])
X = Tongji_375_data.drop(['id', 'admission', 'discharge', 'LDH_first', 'hsCRP_first', 'lymphocytes_first'], axis = 1)

In [12]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)

In [13]:
svm = SVC()
svm.fit(X_train, y_train)

SVC()

In [14]:
X_train_val=pd.concat((X_train,X_test))
y_train_val=np.concatenate((y_train,y_test), axis=0)

In [15]:
cross_val_score(svm, X_train_val, y_train_val, scoring='accuracy', cv = 20)

array([1.        , 1.        , 1.        , 0.88888889, 1.        ,
       0.94444444, 0.94444444, 1.        , 0.88888889, 1.        ,
       0.94444444, 0.88235294, 1.        , 1.        , 0.88235294,
       1.        , 1.        , 1.        , 0.94117647, 1.        ])